In [23]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

Logger issue: Internal C++ object (QTextEditLogger) already deleted.


In [25]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
readout = RouteMeander(design, 'readout',  Dict(
            total_length='6 mm',
            hfss_wire_bonds = True,
            fillet='90 um',
            lead = dict(start_straight='100um'),
            pin_inputs=Dict(
                start_pin=Dict(component='Q1', pin='readout'),
                end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [26]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qres = EPRanalysis(design, "hfss")

In [27]:
hfss = eig_qres.sim.renderer

In [28]:
hfss.start()

INFO 04:44PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:44PM [load_ansys_project]: 	Opened Ansys App
INFO 04:44PM [load_ansys_project]: 	Opened Ansys Desktop v2022.2.0
INFO 04:44PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/CJZ/Documents/Ansoft/
	Project:   Project4
INFO 04:44PM [connect_design]: No active design found (or error getting active design).
INFO 04:44PM [connect]: 	 Connected to project "Project4". No design detected


True

In [29]:
hfss.activate_ansys_design("TransmonReadout", 'eigenmode')  # use new_ansys_design() to force creation of a blank design

04:44PM 43s WARNING [activate_ansys_design]: The design_name=TransmonReadout was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:44PM [connect_design]: 	Opened active design
	Design:    TransmonReadout [Solution type: Eigenmode]
WARNING 04:44PM [connect_setup]: 	No design setup detected.
WARNING 04:44PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [30]:
hfss.render_design(['Q1', 'readout', 'open_to_ground'], [])

In [31]:
hfss.close()

True